In [1]:
from gurobipy import *
from math import *
from random import randint
import sys
import random
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from openpyxl.workbook import Workbook

In [2]:
################################################################
## INPUT FILES
################################################################
#instances = range(1,11) # instance number
network = '43' #number of links or name of the network

network_file = 'Network' + network + '.xlsx'
inst_file = 'Instance26week.xlsx'
arrivals = 'admit.xlsx'
tot_days = 26*7

In [3]:
################################################################
## READ IN DATA FROM FILES
################################################################
df_I = pd.read_excel(inst_file, sheet_name='ICU') # data frame of ICU
df_J = pd.read_excel(inst_file, sheet_name='DSU') # data frame of DSU
df_G = pd.read_excel(network_file, sheet_name='Network') # data frame network
df_A = pd.read_excel(arrivals) # data frame arrivals

I = df_I['ICU name'].tolist() # ICU list
J = df_J['DSU name'].tolist() # DSU list

C_bar = df_I.set_index('ICU name')['Capacity'].to_dict() # total ICU capacity
P_init = df_I.set_index('ICU name')['O1'].to_dict() # initial patient occupancy at ICU

D_bar = df_J.set_index('DSU name')['Capacity'].to_dict() # total DSU capacity
D_prime_bar = df_J.set_index('DSU name')['Variable capacity'].to_dict() # total DSU flexible capacity
Q_init = df_J.set_index('DSU name')['O1'].to_dict() # initial occupancy at DSU
Q_prime_init = df_J.set_index('DSU name')['O1_vc'].to_dict()

Ji = {} # list of feasible DSUs for each ICU
for i in I:
    df_G_i = df_G[df_G['ICU'] == i]
    Ji[i] = df_G_i['DSU'].tolist()

Ij = {} # list of feasible ICUs for each DSU
for j in J:
    df_G_j = df_G[df_G['DSU'] == j]
    Ij[j] = df_G_j['ICU'].tolist()

[u'10W', u'10E', u'RPRE', u'8D', u'5D', u'10D', u'11N', u'12S', u'7F', u'4D', u'9G', u'12N', u'5G', u'R7SN', u'7D', u'10S', u'6D', u'3E', u'12D', u'9N', u'8G', u'10G', u'7G', u'10N', u'9D', u'8W', u'8N', u'R7SS'] 28


In [4]:
tempJ = []
for j in J:
    if len(Ij[j]) > 0:
        tempJ.append(j)
J = tempJ

[u'8D', u'5D', u'10D', u'11N', u'12S', u'7F', u'4D', u'9G', u'12N', u'5G', u'7D', u'10S', u'6D', u'3E', u'12D', u'10G', u'10N', u'9D'] 18


In [5]:
################################################################
## PARAMETERS
################################################################

b = {} # Price for flexible capacity
for j in J:
    b[j] = 0.001 # for simple example
    
l = {} # Initial guess for lambda prices out of ICUs
for i in I:
    l[i] = 0.1
    
iter = 100 # Loop iterations
step = 0.001

In [6]:
def U(a,y):
    ''' Utility function
        '''
    return 1-exp(-a*y)

def ICU(a,C,lam):
    ''' A function that solves the ICU problem and returns the
        value of y
        '''
    y = {}
    y0 = {}
    y1 = {}
    y2 = {}
    y3 = {}
    for i in I:
        if lam[i] <= 0:
            y[i] = C[i]
        else:
            y0[i] = 0
            y3[i] = C[i]
            stat_point = (-1/float(a[i]))*log(lam[i]/float(a[i]))
            y1[i] = floor(stat_point)
            y2[i] = ceil(stat_point)
            Obj = {}
            Obj[y0[i]] = U(a[i], y0[i]) - lam[i]*y0[i]
            Obj[y3[i]] = U(a[i], y3[i]) - lam[i]*y3[i]
            if y1[i] >= 0 and y1[i] <= C[i]:
                Obj[y1[i]] = U(a[i], y1[i]) - lam[i]*y1[i]
            if y2[i] >= 0 and y2[i] <= C[i]:
                Obj[y2[i]] = U(a[i], y2[i]) - lam[i]*y2[i]
            y[i] = max(Obj.iteritems(), key=operator.itemgetter(1))[0]
    return y

def dualobj(a,x,z,y,lam):
    ''' A function calculating the dual objective value for a given
    price and solutions x, z, y
    '''
    v = sum(U(a[i],y[i]) - lam[i]*y[i] for i in I) + sum(sum(lam[i]*x[i,j] + (lam[i]-b[j])*z[i,j] for j in Ji[i]) for i in I)
    return v

def netLP(C,D,D_prime,lam,nu):
    ''' A function that solves the network LP for a given price vector p and returns the
        values of x and z'''
    m = Model()
    x = {}
    z = {}
    d = {}
    for i in I:
        for j in Ji[i]:
            x[i,j] = m.addVar(lb = 0, ub = C[i], vtype = GRB.CONTINUOUS, name = "x[%s,%s]" % (i,j))
            z[i,j] = m.addVar(lb = 0, ub = C[i], vtype = GRB.CONTINUOUS, name = "z[%s,%s]" % (i,j))
    
    m.update()
    m.setObjective(sum(sum(lam[i]*x[i,j] + (lam[i]-b[j])*z[i,j] for j in Ji[i]) for i in I), GRB.MAXIMIZE)
    for j in J:
        m.addConstr(sum(x[i,j] for i in Ij[j]) <= D[j])
        m.addConstr(sum(z[i,j] for i in Ij[j]) <= D_prime[j])

    m.addConstr(sum(sum(z[i,j] for j in Ji[i]) for i in I) <= nu)

    m.optimize()
    xs = {}
    zs = {}
    for i in I:
        for j in Ji[i]:
            xs[i,j] = m.getVarByName('x[%s,%s]' % (i,j)).x
            zs[i,j] = m.getVarByName('z[%s,%s]' % (i,j)).x
    return xs, zs

In [7]:
def findLambda(a,nu,C,D,D_prime):
    # Subgradient algorithm 
    grad = {}
    pr = {}
    xk = {}
    zk = {}
    yk = {}
    objk = {}

    pr[0] = l #initial guess
    xk[0], zk[0] = netLP(C,D,D_prime,pr[0],nu)
    yk[0] = ICU(a,C,pr[0])

    grad[0] = {}
    for i in I:
        grad[0][i] = sum((xk[0][i,j] + zk[0][i,j]) for j in Ji[i]) - yk[0][i]

    objk[0] = dualobj(a,xk[0],zk[0],yk[0],pr[0])
    q_best = objk[0]

    iter_tobest = 0

    p_best = pr[0]
    x_best = xk[0]
    z_best = zk[0]
    y_best = yk[0]


    for k in range(1,iter+1):

        # Candidate price, x, z, and y
        p = {}
        for i in I:
            p[i] = pr[k-1][i] - (step/sqrt(k))*grad[k-1][i]

        x, z = netLP(C,D,D_prime,p,nu)
        y = ICU(a,C,p)
        g = {}
        for i in I:
            g[i] = sum((x[i,j] + z[i,j]) for j in Ji[i]) - y[i]

        q = dualobj(a,x,z,y,p)

        pr[k] = p
        xk[k] = x
        zk[k] = z
        yk[k] = y
        grad[k] = g
        objk[k] = q

        if q <= q_best: # if improvement, update
            q_best = objk[k]
            p_best = pr[k]
            x_best = xk[k]
            z_best = zk[k]
            y_best = yk[k]
            iter_tobest = k
    
    return p_best



In [8]:
##################################################################
def policyOPT(a,nu,C,D,D_prime):
    
    u = {}
    for i in I:
        for k in range(C[i]+1):
            u[i,k] = 1-exp(-a[i]*k)
    
    m = Model()
    x = {}
    z = {}
    y = {}
    for i in I:
        for j in Ji[i]:
            x[i,j] = m.addVar(lb = 0, vtype = GRB.INTEGER, name = "x[%s,%s]" % (i,j))
            z[i,j] = m.addVar(lb = 0, vtype = GRB.INTEGER, name = "z[%s,%s]" % (i,j))
        for k in range(C[i]+1):
            y[i,k] = m.addVar(lb = 0, vtype = GRB.BINARY, name = "y[%s,%s]" % (i,k))
            
    m.update()
    m.setObjective(sum(sum(u[i,k]*y[i,k] for k in range(C[i]+1)) for i in I) - sum(b[j]*sum(z[i,j] for i in Ij[j]) for j in J), GRB.MAXIMIZE)
        
    for i in I:
        m.addConstr(sum(x[i,j]+z[i,j] for j in Ji[i]) == sum(k*y[i,k] for k in range(C[i]+1)))
        m.addConstr(sum(y[i,k] for k in range(C[i]+1)) == 1)
    
    for j in J:
        m.addConstr(sum(x[i,j] for i in Ij[j]) <= D[j])
        m.addConstr(sum(z[i,j] for i in Ij[j]) <= D_prime[j])
    m.addConstr(sum(sum(z[i,j] for j in Ji[i]) for i in I) <= nu)

    m.optimize()
    xs = {}
    zs = {}
    ys = {}

    for i in I:
        for j in Ji[i]:
            xs[i,j] = m.getVarByName('x[%s,%s]' % (i,j)).x
            zs[i,j] = m.getVarByName('z[%s,%s]' % (i,j)).x
        for k in range(C[i]+1):
            ys[i,k] = m.getVarByName('y[%s,%s]' % (i,k)).x

    return xs, zs

In [9]:
C_inst = {}
A_icu = {}
A_dsu = {}
Disch = {}
Disch_vc = {}
x = {}
z = {}

for day in range(1,tot_days+1):
    C_inst[day] = df_I.set_index('ICU name')['C'+str(day)].to_dict()
    A_icu[day] = df_I.set_index('ICU name')['A'+str(day)].to_dict()
    A_dsu[day] = df_J.set_index('DSU name')['A'+str(day)].to_dict()
    Disch[day] = df_J.set_index('DSU name')['C'+str(day)].to_dict()
    Disch_vc[day] = df_J.set_index('DSU name')['C_vc'+str(day)].to_dict()
    
nu_max = 100

In [10]:
################################################################
## TRANSITION FUNCTION
################################################################
def trans_func(day, P0, Q0, Q_prime0, nu0):
    '''A function to determine the transitions from one state to the next
        '''
    # Read transfer requests and new arrivals for the given day
    d = day%7
    if d == 0:
        d = 7
    a = {}
    for i in I:
        a[i] = df_A.loc[i, d]/10.0

    C = {}
    for i in I:
        C[i] = int(min(C_inst[day][i], P0[i]))    
    D = {}
    D_prime = {}
    for j in J:
        D[j] = D_bar[j]-Q0[j]
        D_prime[j] = D_prime_bar[j]-Q_prime0[j]
    #lam = findLambda(a,nu0,C,D,D_prime)
    
    # 1) Make transfer decisions given the state
    x, z = policyOPT(a,nu0,C,D,D_prime)
    
    # 2) Update the ICU populations by removing transfers and admitting new arrivals
    P1 = {}
    lost_icu = {}
    for i in I:
        P1[i] = min(P0[i]-sum(x[i,j]+z[i,j] for j in Ji[i])+A_icu[day][i], C_bar[i])
        lost_icu[i] = max(0, P0[i]-sum(x[i,j]+z[i,j] for j in Ji[i])+A_icu[day][i] - C_bar[i])
        
    # 3) Update the DSU populations by
        # a) adding transfers b) discharging patients c) admitting new arrivals
    Q1 = {}
    Q1_prime = {}
    lost_dsu = {}

    for j in J:
        # a) add transfers 
        population = Q0[j] + sum(x[i,j] for i in Ij[j])
        population_vc = Q_prime0[j] + sum(z[i,j] for i in Ij[j])
        
        # b) discharge patients
        population = max(0, population - Disch[day][j])
        population_vc = max(0, population_vc - Disch_vc[day][j])
        
        # c) admit new arrivals
        Q1[j] = min(population + A_dsu[day][j], D_bar[j])
        lost_dsu[j] = max(0, population + A_dsu[day][j] - D_bar[j])
        Q1_prime[j] = population_vc
        
        
    nu1 = max(0,nu_max - sum(Q1_prime[j] for j in J))
    
    o = sum(U(a[i], sum(x[i,j] for j in Ji[i])) for i in I) - sum(b[j]*sum(z[i,j] for i in Ij[j]) for j in J)
    
    return P1, Q1, Q1_prime, nu1, x, z, lost_icu, lost_dsu, o


In [11]:
#day = 1 # Monday
P = {}
P[0] = df_I.set_index('ICU name')['O1'].to_dict() 

Q = {}
Q[0] = df_J.set_index('DSU name')['O1'].to_dict() 

Q_prime = {}
Q_prime[0] = df_J.set_index('DSU name')['O1_vc'].to_dict() 

nu = {}
nu[0] = sum(Q_prime[0][j] for j in J) + int((sum(D_prime_bar[j] for j in J))/2)


In [12]:
L_icu = {}
L_dsu = {}
o = {}

for day in range(1,tot_days+1):
    P[day], Q[day], Q_prime[day], nu[day], x[day], z[day], L_icu[day], L_dsu[day], o[day] = trans_func(day, P[day-1], Q[day-1], Q_prime[day-1], nu[day-1])

Academic license - for non-commercial use only
Optimize a model with 71 rows, 203 columns and 432 nonzeros
Variable types: 0 continuous, 203 integer (117 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e-03, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 34 rows and 87 columns
Presolve time: 0.00s
Presolved: 37 rows, 116 columns, 230 nonzeros
Found heuristic solution: objective 6.0586547
Variable types: 0 continuous, 116 integer (73 binary)

Root relaxation: objective 1.113238e+01, 24 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      11.1323834   11.13238  0.00%     -    0s

Explored 0 nodes (24 simplex iterations) in 0.14 seconds
Thread count was 4 (of 4 available processors)

Solution count 3: 11.


*    0     0               0       7.2639730    7.26397  0.00%     -    0s

Explored 0 nodes (64 simplex iterations) in 0.15 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 7.26397 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.263972987992e+00, best bound 7.263972987992e+00, gap 0.0000%
Optimize a model with 71 rows, 201 columns and 428 nonzeros
Variable types: 0 continuous, 201 integer (115 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e-03, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 13 rows and 40 columns
Presolve time: 0.00s
Presolved: 58 rows, 161 columns, 359 nonzeros
Variable types: 0 continuous, 161 integer (104 binary)

Root relaxation: objective 8.956928e+00, 70 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | I

Presolve time: 0.00s
Presolved: 46 rows, 156 columns, 338 nonzeros
Variable types: 0 continuous, 156 integer (118 binary)

Root relaxation: objective 6.527263e+00, 58 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       6.5272635    6.52726  0.00%     -    0s

Explored 0 nodes (58 simplex iterations) in 0.16 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 6.52726 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.527263471354e+00, best bound 6.527263471354e+00, gap 0.0000%
Optimize a model with 71 rows, 201 columns and 428 nonzeros
Variable types: 0 continuous, 201 integer (115 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e-03, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective -0.0000000


  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 23 rows and 75 columns
Presolve time: 0.00s
Presolved: 48 rows, 118 columns, 270 nonzeros
Variable types: 0 continuous, 118 integer (84 binary)

Root relaxation: objective 6.699800e+00, 45 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       6.6997996    6.69980  0.00%     -    0s

Explored 0 nodes (45 simplex iterations) in 0.18 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 6.6998 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.699799636016e+00, best bound 6.699799636016e+00, gap 0.0000%
Optimize a model with 71 rows, 201 columns and 428 nonzeros
Variable types: 0 continuous, 201 integer (115 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e-03, 9e-

Optimize a model with 71 rows, 221 columns and 468 nonzeros
Variable types: 0 continuous, 221 integer (135 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e-03, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 18 rows and 55 columns
Presolve time: 0.00s
Presolved: 53 rows, 166 columns, 354 nonzeros
Found heuristic solution: objective 3.0205314
Variable types: 0 continuous, 166 integer (114 binary)

Root relaxation: objective 8.247264e+00, 71 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       8.2472642    8.24726  0.00%     -    0s

Explored 0 nodes (71 simplex iterations) in 0.20 seconds
Thread count was 4 (of 4 available processors)

Solution count 3: 8.24726 3.02053 0.157695 

Optimal solution found

...


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       7.1527809    7.15278  0.00%     -    0s

Explored 0 nodes (53 simplex iterations) in 0.13 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 7.15278 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.152780854652e+00, best bound 7.152780854652e+00, gap 0.0000%
Optimize a model with 71 rows, 185 columns and 396 nonzeros
Variable types: 0 continuous, 185 integer (99 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e-03, 8e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 37 rows and 105 columns
Presolve time: 0.00s
Presolved: 34 rows, 80 columns, 178 nonzeros
Variable types: 0 continuous, 80 integer (56 binary)

Root relaxation: objective 5.170

Presolved: 52 rows, 142 columns, 323 nonzeros
Variable types: 0 continuous, 142 integer (93 binary)

Root relaxation: objective 8.044241e+00, 58 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       8.0442414    8.04424  0.00%     -    0s

Explored 0 nodes (58 simplex iterations) in 0.14 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 8.04424 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.044241354379e+00, best bound 8.044241354379e+00, gap 0.0000%
Optimize a model with 71 rows, 208 columns and 442 nonzeros
Variable types: 0 continuous, 208 integer (122 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e-03, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 18 ro

Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e-03, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 22 rows and 58 columns
Presolve time: 0.00s
Presolved: 49 rows, 128 columns, 298 nonzeros
Variable types: 0 continuous, 128 integer (95 binary)

Root relaxation: objective 7.087108e+00, 39 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       7.0871077    7.08711  0.00%     -    0s

Explored 0 nodes (39 simplex iterations) in 0.13 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 7.08711 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.087107737474e+00, best bound 7.087107737474e+00, gap 0.0000%
Optimize a model with 71 rows, 192 columns and 410 nonzeros
Variable types: 

Optimize a model with 71 rows, 207 columns and 440 nonzeros
Variable types: 0 continuous, 207 integer (121 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e-03, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 16 rows and 57 columns
Presolve time: 0.00s
Presolved: 55 rows, 150 columns, 331 nonzeros
Variable types: 0 continuous, 150 integer (115 binary)

Root relaxation: objective 6.912709e+00, 78 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       6.9127091    6.91271  0.00%     -    0s

Explored 0 nodes (78 simplex iterations) in 0.13 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 6.91271 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.912709078614e+00, bes


Solution count 2: 8.36137 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.361370750922e+00, best bound 8.361370750922e+00, gap 0.0000%
Optimize a model with 71 rows, 182 columns and 390 nonzeros
Variable types: 0 continuous, 182 integer (96 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-03, 8e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 19 rows and 50 columns
Presolve time: 0.00s
Presolved: 52 rows, 132 columns, 296 nonzeros
Variable types: 0 continuous, 132 integer (87 binary)

Root relaxation: objective 6.475734e+00, 57 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       6.4757336    6.47573  0.00%     -    0s

Explored 0 nodes (57 simplex iterations) in 0.16 seconds
Thread co

In [13]:
cols_icu = ['capacity'] + [str(d)+'_arrival_lost' for d in range(1,tot_days+1)] + [str(d)+'_obj' for d in range(1,tot_days+1)]
out_df_icu = pd.DataFrame(columns=cols_icu, index=I)

for i in I:
    out_df_icu.at[i, 'capacity'] = C_bar[i]
    for day in range(1,tot_days+1):       
        if A_icu[day][i] > 0:
            lost = 100*L_icu[day][i]/A_icu[day][i]
        else:
            lost = 0
        out_df_icu.at[i, str(day)+'_arrival_lost'] = lost
        obj = o[day]
        out_df_icu.at[i, str(day)+'_obj'] = obj

out_df_icu.to_excel("policyOPT_icu_output_N"+str(network)+".xlsx")